In [3]:
import string
import re
import os
import pyemd

from glob import glob
from pycontractions import Contractions

In [4]:
DATA_PATH = 'data'

In [13]:
cont = Contractions(w2v_path=os.path.join(DATA_PATH, 'GoogleNews-vectors-negative300.bin.gz'))
cont.load_models()

In [14]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [52]:
def replace_commands(command_line):
    replacements = {
        '^x (.+)$': 'examine \g<1>',
        '^m$': 'commands',
        '^l$': 'look',
        '^nw$': 'northwest',
        '^sw$': 'southwest',
        '^ne$': 'northeast',
        '^se$': 'southeast',
        '^w$': 'west',
        '^e$': 'east',
        '^n$': 'north',
        '^s$': 'south',
        '^u$': 'up', 
        '^d$': 'down', 
        '^i$': 'inventory',
        '^z$': 'wait',
        '^g$': 'again'
    }

    # handle multiple commands
    commands = command_line.split('.')
    parsed_commands = []
    for command in commands:
        for pattern, repl in replacements.items():
            command = re.sub(repl=repl, pattern=pattern, string=command)
        parsed_commands.append(command)
        
    return ' '.join(parsed_commands)

def parse_transcript(infile, outfile, contractions=None, precise=False, keep_caret=False):
    # convenience var to parse different interpreters
    sleepmask = False
    
    try:
        # Try utf first, or fall back to latin-1
        with open(infile, 'r', encoding='utf-8') as inf, open(outfile, 'w', encoding='utf-8') as outf:
            try:
                content = inf.read()
            except UnicodeDecodeError as e:
                print('Could not parse unicode: ' + e.reason)
                inf = open(infile, 'r', encoding='iso-8859-2') # LATIN-1
                content = inf.read()
            
            dest = []
            game = None
            valid_load = None
            
            # skip everything up to the last load attempt (if none, just assume the game's already loaded)
            # make sure we only consider loads for the same game
            loads = list(re.finditer(string=content, pattern=r'say(?:s)? \(to Floyd\), "(?:load( sleepmask)?) (.+)"', flags=re.IGNORECASE))
            if len(loads) > 0:
                valid_load = loads[0]
                game = valid_load.group(2)
                
                print('Looking for loads for game [{}]'.format(game))
                for load in loads[1:]:
                    if game != load.group(2):
                        break
                    valid_load = load

                
                if valid_load.group(1):
                    sleepmask = True
                
                content = content[valid_load.end():]

            # remove html tags
            content = strip_tags(content)
            # remove version commands 
            content = re.sub(repl='', string=content, pattern=r'say(?:s)? \(to Floyd\), "version".*?Floyd \| >\n', flags=re.IGNORECASE | re.DOTALL)
            # remove starting > character for scene descriptions
            content = re.sub(repl='Floyd | \g<1>\n', string=content, pattern=r'Floyd \| >([^\s].+)', flags=re.IGNORECASE)
            
            # replace external commands with inline ones starting with a special '>>' character 
            # do the same with existing inline commands to prevent inconsistencies
            if not sleepmask:
                content = re.sub(repl='Floyd | > \g<1>\n', string=content, pattern=r'(?:.+) say(?:s)? \(to Floyd\), "(.+)"', flags=re.IGNORECASE)
            else:
                content = re.sub(repl='Floyd | >', string=content, pattern=r'Floyd \| >', flags=re.IGNORECASE)
            
            content = content.split('\n')
            # expand contractions
            if contractions:
                content = contractions.expand_texts(content, precise=precise)

            ##############################################################
            ### From here on, we consider everything part of the game. ###
            ##############################################################
            print('Parsing gameplay...')
            
            scene = []
            command_list = []
            about = False
            command_flag = False
            
            for i,line in enumerate(content):
                # skip everything which does not start with 'Floyd |'
                if not line.startswith('Floyd |'):
                    continue
                                       
                # Strip away about commands
                if about:
                    # Either match the explicit "Q" command, or look for the first empty command prompt
                    if re.match(string=line, pattern=r'^Floyd \| >(?: q)?$', flags=re.IGNORECASE):
                        print('Line [{}], about=False'.format(line))
                        about = False
                    continue
                else:
                    if re.match(string=line, pattern=r'^Floyd \| > (about|help)$', flags=re.IGNORECASE):
                        print('Line [{}], about=True'.format(line))
                        about = True
                        continue
                        
                # Just strip blank prompts and "press <key> to continue" lines
                # We need blank lines for this, that's why it's after the about parser
                if re.match(string=line, pattern=r'^Floyd \| >$|^Floyd \|$'):
                    continue
                if re.match(string=line, pattern=r'press (?:.+) to continue', flags=re.IGNORECASE):
                    continue
                    
                #########################
                #######   SCENE   #######
                #########################
                match = re.match(string=line, pattern=r'^Floyd \| ([^>\n].+)', flags=re.IGNORECASE)
                if match:
                    scene.append(match.group(1).strip().lower())
                
                ############################
                #######   COMMAND   ########
                ############################
                match = re.match(string=line, pattern=r'^Floyd \| > (.+)', flags=re.IGNORECASE)        
                if match:
                    command_list.append(match.group(1).strip().lower())
                elif len(command_list) > 0:
                    if len(command_list) > 1:
                        print('Consecutive commands found, joining them {}'.format(command_list))
                        
                    command_line = ' '.join([replace_commands(command.strip()) for command in command_list])
                    command_list = []
                    
                    if keep_caret:
                        command_line = '> ' + command_line
                        
                    newline = '{}\n{}\n'.format(' '.join(scene).replace('  ', ' '), command_line)
                    dest.append(newline)
                    scene = []

            # Append the very last scene
            if len(scene) > 0:
                dest.append(' '.join(scene).replace('  ', ' '))

            outf.writelines(dest)
            del content
            del dest
            print('Done.')
    finally:
        inf.close()

In [55]:
transcripts = sorted(glob('data/transcripts/*.txt'))
for transcript in transcripts:
    print('Parsing file: [{}]'.format(transcript))
    parsed = 'data/parsed/parsed-{}'.format('-'.join(transcript.split('-')[1:]))
    parse_transcript(transcript, parsed, contractions=cont, precise=True, keep_caret=False)

Parsing file: [data/transcripts/raw-12heads.txt]
Looking for loads for game [baptist]
Parsing gameplay...
Done.
Parsing file: [data/transcripts/raw-1893.txt]
Looking for loads for game [1893]
Parsing gameplay...
Consecutive commands found, joining them ['g', 'press yellow enter button']
Consecutive commands found, joining them ['save', 'cf1']
Consecutive commands found, joining them ['save', 'cf2']
Consecutive commands found, joining them ['save', 'cf3']
Consecutive commands found, joining them ['save', 'cf4']
Done.
Parsing file: [data/transcripts/raw-20160221-thesueno-utf8.txt]
Looking for loads for game [thesueno]
Parsing gameplay...
Consecutive commands found, joining them ['saveok.', 'save']
Consecutive commands found, joining them ['saveok.', 'se']
Consecutive commands found, joining them ['saveok.', 'take chair']
Consecutive commands found, joining them ['saveok.', 'dream steeple']
Consecutive commands found, joining them ['saveok.', 's']
Consecutive commands found, joining them 

Floyd | "Yeah, but it is part of a test. See, there is this new agency that is
Floyd | renting out sidekicks. We are going to write a review about it. Oh,
Floyd | here he is now."
Floyd | Into your office walks a goblin: a green humanoid creature with pointy
Floyd | ears, about a meter tall. "Hi, you must be my new boss," he says to
Floyd | you.
Floyd | "I guess I am. What is it you are supposed to do, exactly?"
Floyd | "Well, I am very brave and helpful, plus I am a ferocious killing
Floyd | machine. If you see any kind of monster, or anything at all, and you
Floyd | want it killed, just tell me, and it is toast."
Floyd | "Okay... well, I think we should get going."
Floyd |
Floyd | You and your new companion are off to Treasure Cave Mountain. Your
Floyd | mission is to return with one really big treasure. Any really big
Floyd | treasure.
Floyd |
Floyd | If you are unfamiliar with interactive fiction, type HELP.
Floyd | If you care, type CREDITS.
Floyd | In this game you cannot examine

Consecutive commands found, joining them ['save', 'cf6']
Consecutive commands found, joining them ['x hourglass', 'undo']
Done.
Parsing file: [data/transcripts/raw-afflicted.txt]
Looking for loads for game [afflicted]
Parsing gameplay...
Line [Floyd | > about], about=True
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |  > Introduction (if you are new to this game)
Floyd |    General Instructions (if you are new to IF)
Floyd |    About the Author
Floyd |    Credits
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd |
Floyd | Afflicted
Floyd | A short work of interactive horror
Floyd | Copyright 2008, Doug Egan
Floyd | Annual Competition Version
Floyd |
Floyd | This game is intended to be enjoyed by both newcomers to IF and
Floyd | veteran players.  If you have never played interactive fiction, you
Floyd | are encouraged to return to the main menu and review the information
Floyd | under "General Instructions".
Floyd |
Floyd | Afflicted implements only 

Consecutive commands found, joining them ['knock on door', 'climb rope']
Consecutive commands found, joining them ['save', 'cf1']
Consecutive commands found, joining them ['save', 'i']
Consecutive commands found, joining them ['save', 'cf1']
Consecutive commands found, joining them ['ask man about book', 'ask man about painting']
Consecutive commands found, joining them ['undo', 'l']
Consecutive commands found, joining them ['undo', 'full']
Consecutive commands found, joining them ['undo', '/undo', '/undo', 'restore', 'cf1']
Consecutive commands found, joining them ['l', 'l']
Consecutive commands found, joining them ['get ladder', 's']
Consecutive commands found, joining them ['save', 'cf2']
Consecutive commands found, joining them ['save', 'cf3']
Done.
Parsing file: [data/transcripts/raw-beingandrewplotkin.txt]
Looking for loads for game [beingandrewplotkin]
Parsing gameplay...
Consecutive commands found, joining them ['save', 'cf1']
Consecutive commands found, joining them ['get zarf

Consecutive commands found, joining them ['a', 'of course, of course.  the officers and crew of a royal navy ship are']
Consecutive commands found, joining them ['c', 'thirteen is about the average age for a new midshipman.']
Consecutive commands found, joining them ['e', 'very well then.  what is your surname?']
Consecutive commands found, joining them ['d', 'excellent.  as the game begins, your officer may be described as']
Consecutive commands found, joining them ['e', '"run out the gun!" you yell.  as the gun carriage bounces on the hull']
Consecutive commands found, joining them ['a', '"i am.  acting-lieutenant edward rockersuke of h.m.s. courageous."']
Consecutive commands found, joining them ['d', 'you laugh as he finishes his request.  "you are a prisoner like any']
Consecutive commands found, joining them ['a', '"thank you, sir.  thank you."']
Consecutive commands found, joining them ['c', 'perhaps you are right.']
Consecutive commands found, joining them ['stats', 'name: mr. 

...

Consecutive commands found, joining them ['you', 'you']
Consecutive commands found, joining them ['se', 'se']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['nse', 'nse']
Consecutive commands found, joining them ['se', 'se']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['take brick', 'take brick']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['answer "well"', 'answer "well"']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['take pearl', 'take 

Consecutive commands found, joining them ['ne', 'ne']
Consecutive commands found, joining them ['sw', 'sw']
Consecutive commands found, joining them ['nw', 'nw']
Consecutive commands found, joining them ['se', 'se']
Consecutive commands found, joining them ['sw', 'sw']
Consecutive commands found, joining them ['sw', 'sw']
Consecutive commands found, joining them ['give candies to lizard', 'give candies to lizard']
Consecutive commands found, joining them ['unlock door with key', 'unlock door with key']
Consecutive commands found, joining them ['open door', 'open door']
Consecutive commands found, joining them ['save', 'cf33', 'save']
Consecutive commands found, joining them ['saveok.', 'l', 'l']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['i', 'i']
Consecutive commands found, joining them ['examine notes', 'examine notes']
Consecutive commands found, joining them ['e', 'e']
Consecutive

Consecutive commands found, joining them ['examin serpent', 'examin serpent']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['take sword', 'take sword']
Consecutive commands found, joining them ['take elvish sword', 'take elvish sword']
Consecutive commands found, joining them ['open trophy', 'open trophy']
Consecutive commands found, joining them ['break trophy', 'break trophy']
Consecutive commands found, joining them ['l', 'l']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['n', 'n']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['ne', 'ne']
Consecutive commands found, joining them ['i', 'i']
Consecutive commands found, joining them ['s', '

Consecutive commands found, joining them ['walk through north wall', 'walk through north wall']
Consecutive commands found, joining them ['walk through south door', 'walk through south door']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['d', 'd']
Consecutive commands found, joining them ['l', 'l']
Consecutive commands found, joining them ['ne', 'ne']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['enter wall', 'enter wall']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['ne', 'ne']
Consecutive commands found, joining them ['walk through north wall', 'walk through north wall']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['w', 'w']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['s', 's']
Consecutive co

Consecutive commands found, joining them ['saveok.', 'put collar on cerberus', 'put collar on cerberus']
Consecutive commands found, joining them ['e', 'e']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['open door', 'open door']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['s', 's']
Consecutive commands found, joining them ['examine writing', 'examine writing']
Consecutive commands found, joining them ['score', 'score']
Consecutive commands found, joining them ['wave wand at me', 'wave wand at me']
Consecutive commands found, joining them ['i', 'i']
Consecutive commands found, joining them ['l', 'l']
Consecutive commands found, joining them ['filfre', 'filfre']
Consecutive commands found, joining them ['examine writing', 'examine writing']
Consecutive commands found, joining them ['examine crypt', 'examine crypt']
Consecutive commands found, joining them ['examine south wall', 'examine sout

## Corpus building
Here we build a single corpus of `<scene_before, command, scene_after>` triplets to be used by the model.
We save the model in h5py format for ease-of-use and perfomance reasons.